In [1]:
import PIL.ImageGrab
import pyautogui
import matplotlib.pyplot as plt
import numpy as np
import time

In [2]:
points = {"Wizard":(770,680),
            "Heli" : (580,520),
            "Home" : (830,820),
            "Restart1" : (1085,810),
            "Restart2" : (1130,720),
            "ClearMonkey" : (1750,820),
            "CleanClick" : (600,100),
            "HeliStay" : (410,400),
            "Start" : (1815,1005),
            "Upgrade1" : (1480,490),
            "Upgrade2" : (1480,635),
            "Upgrade3" : (1480,780),
         }
keys = {"Wizard":'a',
         "Heli" : 'b',
        "Upgrade1" : ",",
        "Upgrade2" : ".",
        "Upgrade3" : "/",
        "Clear" : "Esc",
        "Start" : "space",
        "Ability1" : "1",
         }

In [1]:
def getSnap(debug=False):
    im = PIL.ImageGrab.grab()
    if (debug):
        img = np.array(im)
        plt.imshow(img)
        plt.show()
    print(img.shape)
    return np.array(im)

def getSquare(point,width = 100,height=100,debug=False):
    centerX,centerY = point
    im = PIL.ImageGrab.grab()
    img = np.array(im)
    img = img[(centerY-height):(centerY+height),(centerX-width):(centerX+width)]
    if (debug):
        plt.imshow(img)
        plt.show()
    return img

getSnap(True)
# getSquare((165,380),debug=True).shape
# getSquare((410,400),debug=True).shape
# np.mean(getSquare((1815,1005),width=10,height=10,debug=True))
print(np.mean(getSquare((1480,635),width=20,height=20,debug=True)[:,:,1]))

NameError: name 'PIL' is not defined

In [4]:
def placeMonkey(name,debug=False):
    try:
        if (debug):
            i = 3
            while (i > 0):
                time.sleep(1)
                print("Placing in " + str(i),end='\r')
                i -= 1
            print()
        print("Dropping " + name)
        pyautogui.moveTo(points[name])
        pyautogui.keyDown(keys[name])
        pyautogui.keyUp(keys[name])
        pyautogui.click(points[name])
        pyautogui.moveTo(points["ClearMonkey"])
        return True
    except:
        print("Broke on drop")
    return False
    
    
def gameOver(debug=False):
    img = getSquare(points["Restart1"],width=60,height=60,debug=debug)
    if (np.floor(np.mean(img)) == 173):
        print("Game Over")
        return True
    return False
    
def roundOver(debug=False):
    img = getSquare(points['Start'],width=10,height=10,debug=debug)
    if ( np.floor(np.mean(img)) >= 254):
        print("Round Over")
        return True
    return False   

def checkFF(debug=False):
    img = getSquare(points['Start'],width=10,height=10,debug=debug)
    if ( np.floor(np.mean(img)) <= 190):
        startRound()
        print("^Fast Forwarded")

def startRound(debug=False):
    print("Starting new Round")
    pyautogui.click(points["CleanClick"])
    pyautogui.keyDown(keys["Start"])
    pyautogui.keyUp(keys["Start"])

    
def useAbility(ability,stall=0):
    while (stall > 0):
        time.sleep(1)
        print("Using Ability in " + str(stall),end='\r')
        stall -= 1
    print("\nUsing Ability " + ability)
    pyautogui.click(points["CleanClick"])
    pyautogui.keyDown(keys[ability])
    pyautogui.keyUp(keys[ability])
    
def upgradeMonkey(name,upgrade,debug=False):
    if (debug):
        i = 3
        while (i > 0):
            time.sleep(1)
            print("Upgrading in " + str(i),end='\r')
            i -= 1
        print()
    
    pyautogui.click(points["CleanClick"])
    pyautogui.moveTo(points[name])
    pyautogui.click(points[name])
    time.sleep(.5)
    if (np.mean(getSquare(points[upgrade],width=20,height=20,debug=debug)[:,:,1]) > 210):
        print("Upgrading "  + name + " to " + upgrade)
        pyautogui.keyDown(keys[upgrade])
        pyautogui.keyUp(keys[upgrade])
        pyautogui.click(points["CleanClick"])
        return True
    else:
        print("Can't upgrade: " + name + " "  + upgrade + " "+ str(np.mean(getSquare(points[upgrade],width=20,height=20)[:,:,1])))
        return False

# placeMonkey("Wizard",True)
# placeMonkey("Heli",True)
# upgradeMonkey("Heli","Upgrade2")
# startRound()

In [21]:
def playGame(rnd=5,state=0,debug=False):
    i = 3
    while (i > 0):
        time.sleep(1)
        print("Starting Play in " + str(i),end='\r')
        i -= 1
    print("\nPlayin")
    while (not(gameOver(debug=debug))):
        if (rnd == 5 and state == 0):
            if (placeMonkey("Wizard",debug=debug)):
                state += 1
                print("State: " + str(state))
        elif (rnd == 7 and state == 1):
            if (upgradeMonkey("Wizard","Upgrade2",debug=debug)):
                state+=1
                print("State: " + str(state))
        elif (rnd == 9 and state == 2):
            if (upgradeMonkey("Wizard","Upgrade3",debug=debug)):
                state+=1
                print("State: " + str(state))
        elif (rnd == 13 and state == 3):
            if (upgradeMonkey("Wizard","Upgrade2",debug=debug)):
                state+=1
        elif (rnd == 20 and state == 4):
            if (placeMonkey("Heli",debug=debug)):
                state+=1
        elif (rnd == 21 and state == 5):
            if (upgradeMonkey("Heli","Upgrade2",debug=debug)):
                state+=1
        elif (rnd == 23 and state == 6):
            if (upgradeMonkey("Heli","Upgrade2",debug=debug)):
                state+=1
        elif (rnd == 26 and state == 7):
            if (upgradeMonkey("Heli","Upgrade1",debug=debug)):
                state+=1
        elif (rnd == 27 and state == 8):
            if (upgradeMonkey("Heli","Upgrade1",debug=debug)):
                state+=1
        elif (rnd == 32 and state == 9):
            if (upgradeMonkey("Heli","Upgrade1",debug=debug)):
                state+=1
        elif (rnd == 36 and state == 10):
            if (upgradeMonkey("Wizard","Upgrade2",debug=debug)):
                state+=1
        elif (rnd == 49 and state == 11):
            if (upgradeMonkey("Heli","Upgrade1",debug=debug)):
                state+=1
        elif (rnd == 76 and state == 12):
            if (upgradeMonkey("Heli","Upgrade1",debug=debug)):
                state+=1  
        elif (rnd == 78 and state == 13):
            if (upgradeMonkey("Wizard","Upgrade2",debug=debug)):
                state+=1 
        elif (rnd == 92 and state == 14):
            useAbility("Ability1",3)
            state+=1 
        elif (rnd == 93 and state == 15):
            useAbility("Ability1",4)
            state+=1 
        elif (rnd == 95 and state == 16):
            if (upgradeMonkey("Wizard","Upgrade2",debug=debug)):
                useAbility("Ability1",7)
                state+=1  
        elif (rnd == 96 and state == 17):
            useAbility("Ability1",2)
        elif (rnd == 98 and state == 17):
            useAbility("Ability1",7)
            state+=1  
        elif (rnd == 99 and state == 18):
            useAbility("Ability1",1)
            state+=1  
        elif (rnd == 100):
            useAbility("Ability1",2)
        
        
        
        if (state > 4 and state < 9):
            pyautogui.moveTo(points["HeliStay"])
        
        
        if (roundOver(debug=debug)):
            print("Round ended: " + str(rnd))
            startRound(debug=debug)
            checkFF()
            rnd+=1
        else:
            time.sleep(0.5)
    pyautogui.click(points["Restart1"])
    time.sleep(1)
    pyautogui.click(points["Restart2"])
    playGame()
# playGame()

In [23]:
playGame()
# playGame(rnd=94,state=15)

Starting Play in 1
Playin
Dropping Wizard
State: 1
Round Over
Round ended: 5
Starting new Round
Starting new Round
^Fast Forwarded
Round Over
Round ended: 6
Starting new Round
Upgrading Wizard to Upgrade2
State: 2
Round Over
Round ended: 7
Starting new Round
Round Over
Round ended: 8
Starting new Round
Upgrading Wizard to Upgrade3
State: 3
Round Over
Round ended: 9
Starting new Round
Round Over
Round ended: 10
Starting new Round
Round Over
Round ended: 11
Starting new Round
Round Over
Round ended: 12
Starting new Round
Can't upgrade: Wizard Upgrade2 149.516875
Can't upgrade: Wizard Upgrade2 149.516875
Upgrading Wizard to Upgrade2
Round Over
Round ended: 13
Starting new Round
Round Over
Round ended: 14
Starting new Round
Round Over
Round ended: 15
Starting new Round
Round Over
Round ended: 16
Starting new Round
Round Over
Round ended: 17
Starting new Round
Round Over
Round ended: 18
Starting new Round
Round Over
Round ended: 19
Starting new Round
Dropping Heli
Round Over
Round ended: 20

FailSafeException: PyAutoGUI fail-safe triggered from mouse moving to a corner of the screen. To disable this fail-safe, set pyautogui.FAILSAFE to False. DISABLING FAIL-SAFE IS NOT RECOMMENDED.